In [1]:
import pandas as pd

In [3]:
assessments = pd.read_csv("assessments.csv")
student_assessment = pd.read_csv("studentAssessment.csv")

In [6]:
sa_merged = student_assessment.merge(
    assessments[["id_assessment", "code_module", "code_presentation", "assessment_type"]],
    on="id_assessment",
    how="left")

# Фильтруем только экзамены и успешные сдачи (>= 40 баллов)
passed_exams = sa_merged[
    (sa_merged["assessment_type"] == "Exam") &
    (sa_merged["score"] >= 40)
]

# Определяем количество уникальных курсов (module+presentation), которые сдал каждый студент
student_passed_courses = (
    passed_exams
    .groupby("id_student")[["code_module", "code_presentation"]]
    .nunique()
    .max(axis=1)        # берём максимум из nunique по двум колонкам = количество уникальных курсов
)

# Считаем студентов, у которых ровно один сданный курс
students_passed_one_course = (student_passed_courses == 1).sum()

print(students_passed_one_course)

3802


In [14]:
sa_merged = student_assessment.merge(
    assessments[["id_assessment", "code_module", "code_presentation", "assessment_type"]],
    on="id_assessment",
    how="left")

# Фильтруем только экзамены
exams = sa_merged[sa_merged["assessment_type"] == "Exam"].copy()

# Считаем завершённость по каждому экзамену
exam_stats = (
    exams
    .groupby(["id_assessment"])
    .agg(
        total_attempts=("id_student", "count"),
        passed_attempts=("score", lambda s: (s >= 40).sum())
    )
    .reset_index()
)

exam_stats["completion_pct"] = (
    exam_stats["passed_attempts"] / exam_stats["total_attempts"] * 100
).round(2)

# Сортировка внутри курса по возрастанию завершённости
exam_sorted = exam_stats.sort_values(
    by=["completion_pct", "id_assessment"],
    ascending=[True, True]
).reset_index(drop=True)

print(exam_sorted.head(10))

   id_assessment  total_attempts  passed_attempts  completion_pct
0          25340             602            504.0           83.72
1          24299            1168           1019.0           87.24
2          25368             950            842.0           88.63
3          24290             747            664.0           88.89
4          25354             968            878.0           90.70
5          25361             524            485.0           92.56


In [15]:
student_registration = pd.read_csv("studentRegistration.csv")

registrations_by_subject = (
    student_registration
    .groupby("code_module")["id_student"]
    .count()
    .reset_index(name="registrations"))

top3_subjects = registrations_by_subject.sort_values(
    by="registrations",
    ascending=False).head(3)

print(top3_subjects)

  code_module  registrations
1         BBB           7909
5         FFF           7762
3         DDD           6272


In [17]:
import pandas as pd

# Загружаем регистрации
df = pd.read_csv("studentRegistration.csv")

# Флаг отмены регистрации (отток)
df["churn_flag"] = df["date_unregistration"].notna().astype(int)

# Считаем ЧИСЛО отмен по каждому предмету и берём ТОП-3
top3_churn = (
    df.groupby("code_module")["churn_flag"]
      .agg(churned="sum", total_registrations="count")
      .reset_index()
      .sort_values(["churned", "total_registrations"], ascending=[False, False])
      .head(3)
)

print(top3_churn)

  code_module  churned  total_registrations
5         FFF     2380                 7762
1         BBB     2377                 7909
3         DDD     2235                 6272


In [21]:
sa = student_assessment.merge(
    assessments[["id_assessment", "code_presentation", "assessment_type"]],
    on="id_assessment",
    how="left"
)

# Оставляем только экзамены
exams = sa[sa["assessment_type"] == "Exam"].copy()

# Флаг успешной сдачи
exams["success"] = (exams["score"] >= 40).astype(int)

# Фильтруем 2013–2014 по году (первые 4 символа в code_presentation)
exams["year"] = exams["code_presentation"].str[:4].astype(int)
exams = exams[exams["year"].between(2013, 2014)]

# Завершаемость по каждому семестру = успешные попытки / все попытки
sem_completion = (
    exams.groupby("code_presentation")["success"]
         .mean()
         .reset_index(name="completion_rate")
)

# Семестр с минимальной завершаемостью
lowest_semester = sem_completion.sort_values("completion_rate").head(1)

print(lowest_semester)

  code_presentation  completion_rate
0             2013B         0.837209


In [23]:
sa = student_assessment.merge(
    assessments[["id_assessment", "code_module", "code_presentation", "assessment_type"]],
    on="id_assessment", how="left"
)

# Первая успешная сдача Exam по каждому курсу (студент × модуль × семестр)
passed_exams = sa[(sa["assessment_type"] == "Exam") & (sa["score"] >= 40)]
first_pass = (
    passed_exams.sort_values("date_submitted")
    .groupby(["id_student", "code_module", "code_presentation"], as_index=False)
    .first()[["id_student", "code_module", "code_presentation", "date_submitted"]]
    .rename(columns={"date_submitted": "first_pass_day"})
)

# Срок сдачи от старта семестра = день первой успешной сдачи
first_pass["time_to_pass_days"] = first_pass["first_pass_day"]

# Фильтр семестров 2013–2014
first_pass["year"] = first_pass["code_presentation"].str[:4].astype(int)
fp_2013_2014 = first_pass[first_pass["year"].between(2013, 2014)]

# Средний срок сдачи по семестрам и семестр с максимумом
sem_ttp = (
    fp_2013_2014.groupby("code_presentation")["time_to_pass_days"]
    .mean().reset_index(name="avg_time_to_pass_days")
)

# Ответ: семестр с самыми долгими средними сроками сдачи
answer = sem_ttp.loc[sem_ttp["avg_time_to_pass_days"].idxmax(), "code_presentation"]
print(answer)

2014J


In [24]:
sa = student_assessment.merge(
    assessments[["id_assessment", "assessment_type"]],
    on="id_assessment", how="left"
)

# Оставляем только экзамены
exams = sa[sa["assessment_type"] == "Exam"].copy()

# R = среднее время сдачи одного экзамена (средний date_submitted по Exam) для каждого студента
r_per_student = (
    exams.groupby("id_student")["date_submitted"]
         .mean()
         .reset_index(name="R")
)

# Минимальная граница по recency (целое число)
recency_min = int(r_per_student["R"].min())
print(recency_min)

229


In [25]:
# Соединяем с таблицей оценивания, чтобы выделить экзамены
sa = student_assessment.merge(
    assessments[["id_assessment", "assessment_type"]],
    on="id_assessment", how="left"
)

# Оставляем только экзамены
exams = sa[sa["assessment_type"] == "Exam"].copy()

# R = среднее время сдачи экзамена для каждого студента
r_per_student = (
    exams.groupby("id_student")["date_submitted"]
         .mean()
         .reset_index(name="R")
)

# Максимальная граница по recency (целое число)
recency_max = int(r_per_student["R"].max())
print(recency_max)

285


In [26]:
# Присоединяем тип оценивания
sa = student_assessment.merge(
    assessments[["id_assessment", "assessment_type"]],
    on="id_assessment", how="left"
)

# Оставляем только экзамены
exams = sa[sa["assessment_type"] == "Exam"].copy()

# M = средний балл за экзамен по каждому студенту
m_per_student = (
    exams.groupby("id_student")["score"]
         .mean()
         .reset_index(name="M")
)

# Минимальная граница по monetary (целое число)
monetary_min = int(m_per_student["M"].min())
print(monetary_min)

0


In [27]:
# Присоединяем тип оценивания
sa = student_assessment.merge(
    assessments[["id_assessment", "assessment_type"]],
    on="id_assessment", how="left"
)

# Оставляем только экзамены
exams = sa[sa["assessment_type"] == "Exam"].copy()

# M = средний балл за экзамен по каждому студенту
m_per_student = (
    exams.groupby("id_student")["score"]
         .mean()
         .reset_index(name="M")
)

# Максимальная граница по monetary (целое число)
monetary_max = int(m_per_student["M"].max())
print(monetary_max)

100


In [30]:
import numpy as np

# 1) Загрузка данных
assessments = pd.read_csv("assessments.csv")
student_assessment = pd.read_csv("studentAssessment.csv")

# 2) Джойн, чтобы получить тип оценивания и оставить только экзамены
sa = student_assessment.merge(
    assessments[["id_assessment", "assessment_type"]],
    on="id_assessment", how="left")
exams = sa[sa["assessment_type"] == "Exam"].copy()

# 3) Метрики R, F, M по студенту
# R — среднее время сдачи одного экзамена (средний date_submitted по Exam)
r = exams.groupby("id_student")["date_submitted"].mean().reset_index(name="R")

# F — завершаемость по попыткам экзаменов: успешные / все * 100
exams["success"] = (exams["score"] >= 40).astype(int)
f = (
    exams.groupby("id_student")["success"]
         .agg(total="count", passed="sum")
         .reset_index()
)
f["F"] = f["passed"] / f["total"] * 100
f = f[["id_student", "F"]]

# M — средний балл за экзамен
m = exams.groupby("id_student")["score"].mean().reset_index(name="M")

# 4) Собираем RFM
rfm = r.merge(f, on="id_student", how="left").merge(m, on="id_student", how="left")

# 5) Биннинг по правилам
recency_median = rfm["R"].median()
rfm["R_bin"] = np.where(rfm["R"] <= recency_median, 2, 1)

rfm["F_bin"] = np.select(
    [rfm["F"] < 50, rfm["F"] < 100],
    [1, 2],
    default=3)

rfm["M_bin"] = np.select(
    [rfm["M"] < 40, rfm["M"] <= 80],
    [1, 2],
    default=3)

# 6) Код кластера и подсчёт 232
rfm["cluster"] = rfm["R_bin"].astype(str) + rfm["F_bin"].astype(str) + rfm["M_bin"].astype(str)
cluster_232_count = int((rfm["cluster"] == "232").sum())

print(cluster_232_count)

1555
